In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest, f_classif

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
import pandas as pd
import glob

# Path to the folder containing the CSV files
folder_path = '/Users/neverintjoker/Downloads/archive-2/'  # Replace with your folder path

# Get a list of all CSV files in the folder
csv_files = glob.glob(folder_path + "*.csv")

# Combine all CSV files into one DataFrame
data = pd.concat([pd.read_csv(file) for file in csv_files], ignore_index=True)

# Preview the combined DataFrame
print(f"Combined dataset shape: {data.shape}")
print(data.head())

Combined dataset shape: (2830743, 79)
    Destination Port   Flow Duration   Total Fwd Packets  \
0                 22             166                   1   
1              60148              83                   1   
2                123           99947                   1   
3                123           37017                   1   
4                  0       111161336                 147   

    Total Backward Packets  Total Length of Fwd Packets  \
0                        1                            0   
1                        2                            0   
2                        1                           48   
3                        1                           48   
4                        0                            0   

    Total Length of Bwd Packets   Fwd Packet Length Max  \
0                             0                       0   
1                             0                       0   
2                            48                      48   
3         

In [7]:
# Check for missing values
print(data.isnull().sum().sum())  # Total missing values

# Drop or fill missing values
data = data.dropna()  # or use data.fillna(method='ffill', inplace=True)

# Separate features and target
X = data.drop(columns=['Label'])  # Drop the target column
y = data['Label']  # Target column

# Convert categorical features to numeric if necessary
X = pd.get_dummies(X, drop_first=True)

1358


KeyError: "['Label'] not found in axis"

In [9]:
print(data.columns)

Index([' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Total Length of Fwd Packets',
       ' Total Length of Bwd Packets', ' Fwd Packet Length Max',
       ' Fwd Packet Length Min', ' Fwd Packet Length Mean',
       ' Fwd Packet Length Std', 'Bwd Packet Length Max',
       ' Bwd Packet Length Min', ' Bwd Packet Length Mean',
       ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s',
       ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min',
       'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max',
       ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std',
       ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags',
       ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length',
       ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s',
       ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean',
       ' Packet Length Std', ' Packet Length Variance', '

In [10]:
# Strip leading/trailing spaces from column names
data.columns = data.columns.str.strip()

# Check updated column names
print(data.columns)

Index(['Destination Port', 'Flow Duration', 'Total Fwd Packets',
       'Total Backward Packets', 'Total Length of Fwd Packets',
       'Total Length of Bwd Packets', 'Fwd Packet Length Max',
       'Fwd Packet Length Min', 'Fwd Packet Length Mean',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Mean',
       'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s',
       'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min',
       'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max',
       'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std',
       'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags',
       'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Min Packet Length', 'Max Packet Length', 'Packet Length Mean',
       'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
       'SYN Flag Co

In [11]:
# Separate features and target
X = data.drop(columns=['Label'])  # Drop the target column
y = data['Label']  # Select the target column

# Verify the separation
print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")

Features shape: (2829385, 78)
Target shape: (2829385,)


In [12]:
from sklearn.feature_selection import SelectKBest, f_classif

# Select top k features
k = 10  # Choose the number of top features to select
selector = SelectKBest(score_func=f_classif, k=k)
X_new = selector.fit_transform(X, y)

# Get scores and feature names
scores = selector.scores_
columns = X.columns

# Combine feature names and scores into a DataFrame
feature_scores = pd.DataFrame({'Feature': columns, 'Score': scores})
feature_scores = feature_scores.sort_values(by='Score', ascending=False)

# Display the top k features
print(feature_scores.head(k))

ValueError: Input X contains infinity or a value too large for dtype('float64').

In [13]:
# Check for infinity or NaN in the dataset
print(f"Infinity values present: {np.isinf(X).values.any()}")
print(f"NaN values present: {np.isnan(X).values.any()}")

# Count the number of infinity and NaN values per column
print(X.replace([np.inf, -np.inf], np.nan).isnull().sum())

Infinity values present: True
NaN values present: False
Destination Port               0
Flow Duration                  0
Total Fwd Packets              0
Total Backward Packets         0
Total Length of Fwd Packets    0
                              ..
Active Min                     0
Idle Mean                      0
Idle Std                       0
Idle Max                       0
Idle Min                       0
Length: 78, dtype: int64


In [14]:
# Replace infinity values with the maximum finite value in the column
X = X.replace([np.inf, -np.inf], np.nan)  # Temporarily replace inf with NaN
X = X.fillna(X.max())  # Replace NaN (formerly inf) with the column's max finite value

In [15]:
# Check again for infinity and NaN values
print(f"Infinity values present: {np.isinf(X).values.any()}")
print(f"NaN values present: {np.isnan(X).values.any()}")

Infinity values present: False
NaN values present: False


In [16]:
from sklearn.feature_selection import SelectKBest, f_classif

k = 10  # Select top 10 features
selector = SelectKBest(score_func=f_classif, k=k)
X_new = selector.fit_transform(X, y)

# Get scores and feature names
scores = selector.scores_
columns = X.columns

# Combine feature names and scores
feature_scores = pd.DataFrame({'Feature': columns, 'Score': scores})
feature_scores = feature_scores.sort_values(by='Score', ascending=False)

# Display top k features
print(feature_scores.head(k))

/Users/neverintjoker/Downloads/archive-2/my_env/lib/python3.13/site-packages/sklearn/feature_selection/_univariate_selection.py:111: UserWarning: Features [31 33 56 57 58 59 60 61] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/Users/neverintjoker/Downloads/archive-2/my_env/lib/python3.13/site-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


                   Feature          Score
13   Bwd Packet Length Std  181460.227419
12  Bwd Packet Length Mean  168847.981073
54    Avg Bwd Segment Size  168847.981073
10   Bwd Packet Length Max  165722.561497
41       Packet Length Std  150706.891124
22             Fwd IAT Std  144486.832108
39       Max Packet Length  133127.634116
42  Packet Length Variance  130666.363683
77                Idle Min  123346.904326
74               Idle Mean  122838.615605


In [17]:
from sklearn.feature_selection import VarianceThreshold

# Remove features with zero variance (constant features)
constant_filter = VarianceThreshold(threshold=0)  # threshold=0 means detect constant features
X = constant_filter.fit_transform(X)

# Update column names after removing constant features
constant_features_idx = constant_filter.get_support(indices=False)
X = pd.DataFrame(X, columns=pd.Index(X.columns)[constant_features_idx])

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [18]:
from sklearn.feature_selection import VarianceThreshold

# Save column names before applying VarianceThreshold
original_columns = X.columns

# Remove constant features
constant_filter = VarianceThreshold(threshold=0)
X_filtered = constant_filter.fit_transform(X)  # This returns a NumPy array

# Get the column names of the features that are retained
retained_columns = original_columns[constant_filter.get_support(indices=True)]

# Convert back to a DataFrame with the updated column names
X = pd.DataFrame(X_filtered, columns=retained_columns)

# Verify the shape
print(f"Shape after removing constant features: {X.shape}")

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [19]:
# Ensure X is a DataFrame with column names from the original data
X = pd.DataFrame(X, columns=data.columns[:-1])  # Adjust for your dataset

ValueError: Shape of passed values is (2829385, 70), indices imply (2829385, 78)

In [20]:
print(f"Shape of X: {X.shape}")
print(f"Number of columns in original data: {data.columns[:-1].shape[0]}")  # Exclude the target column

Shape of X: (2829385, 70)
Number of columns in original data: 78


In [21]:
# If X has fewer columns, filter the original column names
adjusted_columns = data.columns[:-1][:X.shape[1]]  # Ensure names match the shape

In [22]:
# Create the DataFrame with matching column names
X = pd.DataFrame(X, columns=adjusted_columns)
print(f"Shape of X after adjustment: {X.shape}")

Shape of X after adjustment: (2829385, 70)


In [23]:
# Original feature columns (excluding the target column)
original_columns = data.columns[:-1]  

# Adjusted columns in the current X
current_columns = original_columns[:X.shape[1]]

# Check missing columns
missing_columns = set(original_columns) - set(current_columns)
print(f"Missing columns: {missing_columns}")

Missing columns: {'Active Mean', 'Active Std', 'Idle Max', 'Idle Std', 'Active Max', 'Idle Mean', 'Active Min', 'Idle Min'}


In [25]:
# Check if columns are constant
print(data[['Active Mean', 'Active Std', 'Idle Max', 'Idle Std', 
            'Active Max', 'Idle Mean', 'Active Min', 'Idle Min']].nunique())

# Check for infinite values in the missing columns
print(data[['Active Mean', 'Active Std', 'Idle Max', 'Idle Std', 
            'Active Max', 'Idle Mean', 'Active Min', 'Idle Min']].replace([np.inf, -np.inf], np.nan).isnull().sum())

Active Mean    326325
Active Std     202826
Idle Max       149737
Idle Std       197616
Active Max     299565
Idle Mean      222016
Active Min     175670
Idle Min       223888
dtype: int64
Active Mean    0
Active Std     0
Idle Max       0
Idle Std       0
Active Max     0
Idle Mean      0
Active Min     0
Idle Min       0
dtype: int64


In [26]:
# Add the missing columns back to X from the original data
for col in ['Active Mean', 'Active Std', 'Idle Max', 'Idle Std', 
            'Active Max', 'Idle Mean', 'Active Min', 'Idle Min']:
    if col in data.columns:  # Ensure the column exists in the original data
        X[col] = data[col]

# Verify the updated shape of X
print(f"Shape after reintroducing missing columns: {X.shape}")

Shape after reintroducing missing columns: (2829385, 78)


In [27]:
print(len(data.columns))

79


In [28]:
from sklearn.feature_selection import SelectKBest, f_classif

# Select top k features
k = 79  # You can adjust this to experiment with more or fewer features
selector = SelectKBest(score_func=f_classif, k=k)
X_new = selector.fit_transform(X, y)

# Get scores and feature names
scores = selector.scores_
columns = X.columns

# Combine feature names and scores into a DataFrame
feature_scores = pd.DataFrame({'Feature': columns, 'Score': scores})
feature_scores = feature_scores.sort_values(by='Score', ascending=False)

# Display the top k features
print("Top features by ANOVA F-Test:")
print(feature_scores.head(k))

ValueError: Input X contains NaN.
SelectKBest does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [29]:
# Check for NaN values in the dataset
print(X.isnull().sum())

# Count the total number of NaN values
print(f"Total NaN values: {X.isnull().sum().sum()}")

Destination Port                  0
Flow Duration                     0
Total Fwd Packets                 0
Total Backward Packets            0
Total Length of Fwd Packets       0
                               ... 
Idle Std                       1358
Active Max                     1358
Idle Mean                      1358
Active Min                     1358
Idle Min                       1358
Length: 78, dtype: int64
Total NaN values: 10864


In [30]:
# Check the percentage of missing values in each column
missing_percentage = (X.isnull().sum() / len(X)) * 100
print(missing_percentage[missing_percentage > 0])  # Display only columns with NaN values

Active Mean    0.047996
Active Std     0.047996
Idle Max       0.047996
Idle Std       0.047996
Active Max     0.047996
Idle Mean      0.047996
Active Min     0.047996
Idle Min       0.047996
dtype: float64


In [31]:
# Impute missing values with the mean
X = X.fillna(X.mean())

In [32]:
# Check for any remaining NaN values
print(f"Total NaN values after imputation: {X.isnull().sum().sum()}")

Total NaN values after imputation: 0


In [38]:
from sklearn.feature_selection import SelectKBest, f_classif

# Select top k features
k = 78
selector = SelectKBest(score_func=f_classif, k=k)
X_new = selector.fit_transform(X, y)

# Get scores and feature names
scores = selector.scores_
columns = X.columns

# Combine feature names and scores into a DataFrame
feature_scores = pd.DataFrame({'Feature': columns, 'Score': scores})
feature_scores = feature_scores.sort_values(by='Score', ascending=False)

with open("/Users/neverintjoker/Downloads/archive-2/Top 78 features.txt", "w") as file:
    file.write("Top features by ANOVA F-Test:\n")
    file.write(feature_scores.head(k).to_string(index=False))

# Display the top k features
print("Top features by ANOVA F-Test:")
print(feature_scores.head(k))


Top features by ANOVA F-Test:
                   Feature          Score
13   Bwd Packet Length Std  181460.227419
12  Bwd Packet Length Mean  168847.981073
52     Average Packet Size  168847.981073
10   Bwd Packet Length Max  165722.561497
39       Max Packet Length  150706.891124
..                     ...            ...
47          ACK Flag Count       5.512130
61       Bwd Avg Bulk Rate       0.322610
33           Bwd URG Flags       0.133647
32           Fwd URG Flags       0.076232
53    Avg Fwd Segment Size       0.076232

[78 rows x 2 columns]
